In [75]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess

epochs = 70
trainable=True

In [36]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [37]:
from keras_contrib.layers import CRF


In [38]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [39]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [40]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [150]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [151]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12152
2867
3005


In [152]:
print(train_sentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [153]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [154]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [155]:
labelSet = set()
words = {}
characters=set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
            words[token.lower()] = True

In [156]:
print(labelSet)

{'O', 'I-ORG', 'B-LOC', 'B-ORG', 'B-PER', 'B-MISC', 'I-LOC', 'I-MISC', 'I-PER'}


In [157]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [158]:
print(label2Idx)

{'O': 0, 'B-PER': 4, 'I-ORG': 1, 'I-LOC': 6, 'B-LOC': 2, 'B-ORG': 3, 'I-MISC': 7, 'I-PER': 8, 'B-MISC': 5}


In [159]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [160]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/jindal/NER/german_words_conll.txt','wb')
f.write(string_words.encode())
f.close()

In [161]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'allLower': 1, 'numeric': 0, 'contains_digit': 6, 'initialUpper': 3, 'mainly_numeric': 5, 'allUpper': 2, 'other': 4, 'PADDING_TOKEN': 7}


In [162]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings_conll.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [163]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[2]))

300


In [164]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{"'": 0, 'J': 1, 'o': 48, '3': 47, '%': 28, 's': 2, 'P': 5, 'ñ': 91, 'F': 51, 'Ü': 3, 'V': 52, '/': 4, 'U': 49, 'é': 53, '5': 54, 'ç': 6, 'l': 7, '1': 56, ')': 57, 'ü': 58, 'I': 8, 'D': 59, '*': 9, '&': 10, '"': 33, 'Ö': 11, 'C': 61, '.': 86, 'K': 50, 'H': 12, '2': 63, '§': 64, 'è': 13, 't': 14, 'z': 65, 'ô': 15, 'v': 16, 'ò': 55, ';': 17, '=': 66, 'N': 67, 'G': 18, 'M': 68, 'T': 69, 'E': 70, 'h': 71, 'j': 72, 'ß': 19, 'c': 73, '9': 74, 'X': 20, 'ê': 21, 'Z': 75, 'q': 22, 'á': 23, 'y': 24, 'ö': 26, 'w': 77, 'í': 78, '4': 27, '0': 79, ',': 80, '8': 81, 'ó': 29, ':': 30, 'm': 31, 'L': 32, 'R': 34, 'i': 93, '-': 35, 'a': 82, '(': 83, 'Ä': 84, 'e': 36, '6': 37, 'd': 85, 'ä': 38, 'g': 60, 'n': 39, 'S': 87, '?': 40, '!': 89, 'B': 90, 'x': 62, 'Q': 41, 'f': 92, 'r': 42, 'O': 43, 'p': 44, 'Y': 25, 'W': 45, 'k': 94, 'u': 46, '+': 95, 'ì': 96, 'A': 97, 'b': 88, '7': 76}


In [165]:
print(trainSentences[0]) 

[['Ereignis', ['E', 'r', 'e', 'i', 'g', 'n', 'i', 's'], 'O'], ['und', ['u', 'n', 'd'], 'O'], ['Erzählung', ['E', 'r', 'z', 'ä', 'h', 'l', 'u', 'n', 'g'], 'O'], ['oder', ['o', 'd', 'e', 'r'], 'O'], [':', [':'], 'O']]


In [166]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [167]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
# print((train_set[0][2]))

[[10296, 38277, 17527, 29285, 40756], [3, 1, 3, 1, 4], array([[70, 42, 36, 93, 60, 39, 93,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [46, 39, 85,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [70, 42, 65, 38, 71,  7, 46, 39, 60,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [48, 85, 36, 42,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
      

In [168]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [169]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2940        char_input[0][0]                 
__________________________________________________________________________________________________
dropout1 (Dropout)              (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_16 (TimeDistri (None, None, 52, 30) 2730        dropout1[0][0]                   
__________________________________________________________________________________________________
maxpool (T

In [170]:
# for x in range(10):
#     print(i)
maxf1 = 0
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch, epochs))
    a = Progbar(len(train_batch_len))
    for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
        labels, tokens, casing, char = batch
        labels = labels.tolist()
#         print(labels)
        for sentence in labels:
            for i,label in enumerate(sentence):
                temp = [0]*len(label2Idx)
                value = label[0]
                temp[value]=1
                sentence[i] = temp
        labels = np.array(labels)
#         print(labels)
        model.train_on_batch([tokens, casing, char], labels)
        a.update(i)
    if epoch >= 50:
        predLabels, correctLabels = tag_dataset(dev_batch)        
#         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
        predLabels = np.concatenate(predLabels).ravel()
        correctLabels = np.concatenate(correctLabels).ravel()
        f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
        print(f1_dev)
#         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

        if f1_dev > maxf1:
            model.save('german-CNN-BiLSTM-CRF.h5')
            maxf1 = f1_dev
#         predLabels, correctLabels = tag_dataset(dev_batch)        
    #     with open("dropout=0.75.txt",'w') as f:
#         pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
    #     with open('dropout=0.75.txt') as f:
    #         x = str(epoch)+ " "+f1_dev
    #         f.write(x)
    #         f.write('\n')
#         print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))


id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))        



# command = "perl nereval.perl"
# input_file= open(input_file_name)
# output_file_name = '/home/jindal/notebooks/jindal/NER/1cnntest_result.txt'
# output_file=open(output_file_name,'w')
# import subprocess
# process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=r'/home/jindal/notebooks/Resources/GermEVAL')
# out, err = process.communicate()
# #     print(out)
# print('******************************************************************************')

Epoch 0/70
100/79 [=====================================] - 22s 219ms/step
Epoch 1/70
100/79 [=====================================] - 15s 153ms/step
Epoch 2/70
100/79 [=====================================] - 16s 159ms/step
Epoch 3/70
100/79 [=====================================] - 15s 150ms/step
Epoch 4/70
100/79 [=====================================] - 15s 151ms/step
Epoch 5/70
100/79 [=====================================] - 15s 150ms/step
Epoch 6/70
100/79 [=====================================] - 15s 152ms/step
Epoch 7/70
100/79 [=====================================] - 15s 151ms/step
Epoch 8/70
100/79 [=====================================] - 15s 151ms/step
Epoch 9/70
100/79 [=====================================] - 15s 151ms/step
Epoch 10/70
100/79 [=====================================] - 16s 163ms/step
Epoch 11/70
100/79 [=====================================] - 15s 153ms/step
Epoch 12/70
100/79 [=====================================] - 16s 159ms/step
Epoch 13/70
100/79 [==

100/79 [=====================================] - 16s 161ms/step
Epoch 27/70
100/79 [=====================================] - 15s 147ms/step
Epoch 28/70
100/79 [=====================================] - 16s 156ms/step
Epoch 29/70
100/79 [=====================================] - 15s 155ms/step
Epoch 30/70
100/79 [=====================================] - 15s 151ms/step
Epoch 31/70
100/79 [=====================================] - 15s 154ms/step
Epoch 32/70
100/79 [=====================================] - 16s 156ms/step
Epoch 33/70
100/79 [=====================================] - 16s 159ms/step
Epoch 34/70
100/79 [=====================================] - 16s 156ms/step
Epoch 35/70
100/79 [=====================================] - 16s 156ms/step
Epoch 36/70
100/79 [=====================================] - 16s 157ms/step
Epoch 37/70
100/79 [=====================================] - 16s 163ms/step
Epoch 38/70
100/79 [=====================================] - 16s 155ms/step
Epoch 39/70
100/79 [====

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 51/70
2866/2867 [============================>.] - ETA: 0s0.5645129230304237
Epoch 52/70
2866/2867 [============================>.] - ETA: 0s0.5661200863482391
Epoch 53/70
2866/2867 [============================>.] - ETA: 0s0.567513773393331
Epoch 54/70
2866/2867 [============================>.] - ETA: 0s0.5723644129323413
Epoch 55/70
2866/2867 [============================>.] - ETA: 0s0.5620775007873011
Epoch 56/70
2866/2867 [============================>.] - ETA: 0s0.564336325123685
Epoch 57/70
2866/2867 [============================>.] - ETA: 0s0.5700033878101335
Epoch 58/70
2866/2867 [============================>.] - ETA: 0s0.5725668232948908
Epoch 59/70
2866/2867 [============================>.] - ETA: 0s0.571670569329849
Epoch 60/70
2866/2867 [============================>.] - ETA: 0s0.5739262727269984
Epoch 61/70
2866/2867 [============================>.] - ETA: 0s0.5719519954921873
Epoch 62/70
2866/2867 [============================>.] - ETA: 0s0.5731667011355671
Epoch 6

In [171]:
model.load_weights('german-CNN-BiLSTM-CRF.h5')
predLabels, correctLabels = tag_dataset(test_batch)        
# pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
# print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='1cnntest.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
        string = string.encode()
        f.write(string)
        id+=1

3004/3005 [============================>.] - ETA: 0s

In [172]:
predLabels, correctLabels = tag_dataset(dev_batch)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

2866/2867 [============================>.] - ETA: 0sDev-Data: Prec: 0.000, Rec: 0.000, F1: 0.000


In [173]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

3004/3005 [============================>.] - ETA: 0sTest-Data: Prec: 0.000, Rec: 0.000, F1: 0.000


In [70]:
id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))

5099


In [71]:
print(idx2Label)

{0: 'I-PERderiv', 1: 'I-LOCpart', 2: 'I-OTHderiv', 3: 'B-LOCderiv', 4: 'I-ORG', 5: 'I-ORGpart', 6: 'B-OTH', 7: 'B-PER', 8: 'B-LOCpart', 9: 'I-LOC', 10: 'I-ORGderiv', 11: 'B-OTHderiv', 12: 'I-PER', 13: 'B-ORGpart', 14: 'B-ORG', 15: 'I-OTH', 16: 'I-OTHpart', 17: 'B-PERpart', 18: 'O', 19: 'B-PERderiv', 20: 'B-OTHpart', 21: 'B-LOC', 22: 'I-LOCderiv', 23: 'I-PERpart', 24: 'B-ORGderiv'}


In [175]:
id = 1
f = open('1cnntest.txt','wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        if word.isalpha():
            #string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
        
            string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
            string = string.encode()
            f.write(string)
            id+=1
        

In [164]:
model.save('german-CNN-BiLSTM-CRF.h5')